In [2]:
import pandas as pd
import numpy as np
import math
import random
import re
import numexpr as ne 
from operator import itemgetter

In [3]:
word_vectors=pd.read_csv(r'F:\work\12.3 文章关键词\word2vec\model_mincounts_100_title3.csv')
vec_dict={}
for i in word_vectors.index:
    word=word_vectors['word'][i]
    tmp_list=[]
    for j in range(1,101):
        value=word_vectors[str(j)][i]
        tmp_list.append(value)
    vec_dict[word]=np.array(tmp_list)

In [3]:
essay_content=pd.read_csv(r'F:\work\12.3 文章关键词\essay_content.csv')
essay_info={}
for i in essay_content.index:
    itemid=essay_content['itemid'][i]
    title=essay_content['c_title'][i]
    content=essay_content['c_content'][i]
    essay_info.setdefault(itemid,{})
    essay_info[itemid]['title']=title
    essay_info[itemid]['content']=content

In [4]:
key_words=pd.read_csv(r'F:\work\12.3 文章关键词\key_words.csv',encoding='gbk')
key_dict={}
for i in key_words.index:
    itemid=key_words['itemid'][i]
    weights=key_words['weights'][i][1:-1]
    lst=weights.split('), (')
    if lst!=['[]']:
        for k in lst:
            key_str=k.split(',')[0]
            rule=r"'(.*?)'" # 正则规则
            if key_str!='':
                key=re.findall(rule,key_str)[0]
                value=k.split(',')[1]
                if value!='':
                    key_dict.setdefault(itemid,{})
                    value=float(re.sub("[^0-9.]","",value))
                    key_dict[itemid][key]=value

In [5]:
key_words=pd.read_csv(r'F:\work\12.3 文章关键词\key_words.csv',encoding='gbk')
key_dict={}
for i in key_words.index:
    itemid=key_words['itemid'][i]
    tfidf_weights=key_words['tfidf'][i][1:-1]
    textrank_weights=key_words['textrank'][i][1:-1]
    tfidf_lst=tfidf_weights.split('), (')
    textrank_lst=textrank_weights.split('), (')
    tfidf_dict={}
    textrank_dict={}
    if tfidf_lst!=['[]']:
        for k in tfidf_lst:
            key_str=k.split(',')[0]
            rule=r"'(.*?)'" # 正则规则
            if key_str!='':
                key=re.findall(rule,key_str)[0]
                value=k.split(',')[1]
                if value!='':
                    tfidf_dict[key]=float(re.sub("[^0-9.]","",value))
    if textrank_lst!=['[]']:
        for k in textrank_lst:
            key_str=k.split(',')[0]
            rule=r"'(.*?)'" # 正则规则
            if key_str!='':
                key=re.findall(rule,key_str)[0]
                value=k.split(',')[1]
                if value!='':
                    textrank_dict[key]=float(re.sub("[^0-9.]","",value))
    key_dict[itemid]=tfidf_dict
'''
    keys=list(set(list(tfidf_dict.keys())+list(textrank_dict.keys())))
    for key in keys:
        if key not in tfidf_dict.keys() and key in textrank_dict.keys():
            value=0.3*textrank_dict[key]
        if key in tfidf_dict.keys() and key not in textrank_dict.keys():
            value=0.7*tfidf_dict[key]
        if key in tfidf_dict.keys() and key in textrank_dict.keys():
            value=0.7*tfidf_dict[key]+0.3*textrank_dict[key]
        key_dict.setdefault(itemid,{})
        key_dict[itemid][key]=value
'''

'\n    keys=list(set(list(tfidf_dict.keys())+list(textrank_dict.keys())))\n    for key in keys:\n        if key not in tfidf_dict.keys() and key in textrank_dict.keys():\n            value=0.3*textrank_dict[key]\n        if key in tfidf_dict.keys() and key not in textrank_dict.keys():\n            value=0.7*tfidf_dict[key]\n        if key in tfidf_dict.keys() and key in textrank_dict.keys():\n            value=0.7*tfidf_dict[key]+0.3*textrank_dict[key]\n        key_dict.setdefault(itemid,{})\n        key_dict[itemid][key]=value\n'

In [6]:
for essay in essay_info.keys():
    essay_info[essay]['keywords']=[]
    if essay in key_dict.keys():
        essay_info[essay]['keywords']=list(key_dict[essay].keys())

In [7]:
item_vec_dict={}
for item in key_dict.keys():
    for key in key_dict[item].keys():
        if key in vec_dict.keys():
            item_vec_dict.setdefault(item,np.zeros([100,]))
            item_vec_dict[item]+=vec_dict[key]*key_dict[item][key]
    if item in item_vec_dict.keys():
        x=item_vec_dict[item]
        y=np.linalg.norm(x)
        item_vec_dict[item]=list(x/y)

In [8]:
def bit_product_sum(x, y):
    return sum([ne.evaluate('a*b') for a,b in zip(x, y)])

def cosine_similarity(x, y, norm=False):
    """ 计算两个向量x和y的余弦相似度 """
    assert len(x) == len(y), "len(x) != len(y)"
    zero_list = [0] * len(x)
    if x == zero_list or y == zero_list:
        return float(1) if x == y else float(0)

    # method 1
    res = np.array([x[i]*y[i] for i in range(len(x))])
    cos = ne.evaluate('sum(res)')

    # method 2
    # cos = bit_product_sum(x, y) / (np.sqrt(bit_product_sum(x, x)) * np.sqrt(bit_product_sum(y, y)))

    # method 3
    # dot_product, square_sum_x, square_sum_y = 0, 0, 0
    # for i in range(len(x)):
    #     dot_product += x[i] * y[i]
    #     square_sum_x += x[i] * x[i]
    #     square_sum_y += y[i] * y[i]
    # cos = dot_product / (np.sqrt(square_sum_x) * np.sqrt(square_sum_y))

    return 0.5*cos+0.5 if norm else cos  # 归一化到[0, 1]区间内

In [9]:
item_lst=list(item_vec_dict.keys())
vec=item_vec_dict[item_lst[0]]
vecary=np.array(vec).reshape(1,100)
for i in range(1,len(item_lst)):
    vec=item_vec_dict[item_lst[i]]
    vecary=np.insert(vecary,i,values=vec,axis=0)

In [10]:
sim_matrix=np.dot(vecary,vecary.T)
sim_matrix

array([[1.        , 0.45646095, 0.44033306, ..., 0.59281007, 0.54478811,
        0.53704118],
       [0.45646095, 1.        , 0.23946474, ..., 0.2546476 , 0.37761203,
        0.21907875],
       [0.44033306, 0.23946474, 1.        , ..., 0.62859673, 0.4380114 ,
        0.5509099 ],
       ...,
       [0.59281007, 0.2546476 , 0.62859673, ..., 1.        , 0.41212965,
        0.65753058],
       [0.54478811, 0.37761203, 0.4380114 , ..., 0.41212965, 1.        ,
        0.41537544],
       [0.53704118, 0.21907875, 0.5509099 , ..., 0.65753058, 0.41537544,
        1.        ]])

In [11]:
i=82
itemid=item_lst[i]
title=essay_info[itemid]['title']
keywords=essay_info[itemid]['keywords']
print(title)
print(keywords)
print('------'*8)
sorted_list=sorted(enumerate(list(sim_matrix[i])), key=itemgetter(1),reverse=True)
for j,sim in sorted_list[1:11]:
    title=essay_info[item_lst[j]]['title']
    keywords=essay_info[item_lst[j]]['keywords']
    print(title)
    print(keywords)
    print(sim)
    print('------'*8)

山寨品牌被嫌弃？老司机教你选摩托车，车主：十年开不坏
['品牌', '摩托车', '嫌弃', '教你', '山寨', '年代', '十年', '组装', '零件', '短评', '合资', '司机', '很久以前', '技术', '品质', '消费者', '小编', '几千块', '大运', '购买']
------------------------------------------------
国产摩托车品牌到底哪个好，怎么说哪个品牌好的都有，彻底蒙圈了
['品牌', '车型', '国产', '自主', '答案', '梯队', '选择', '品质', '相关', '找到', '不谈', '国产车', '合资', '摩托车', '用户', '这位', '果子', '要说', '分享', '夸赞']
0.8854486382966005
------------------------------------------------
今天看到帖子说国产摩托车和进口摩托车差别到底在哪里？
['厂家', '西安', '帖子', '品控', '国产品牌', '好几年', '海报', '消费', '市场', '小学', '摩托', '低端', '技术', '人群', '差别', '国产', '下乡', '城市', '产品', '那车']
0.878817311098661
------------------------------------------------
有没有国产跨骑摩托车，没用任何国外技术的车型？
['国外', '自主', '技术', '意义', '国产', '车型', '没用', '百分之百', '摩托车', '依然', '更何况', '生产', '大牌', '摆脱', '有没有', '外国', '骑摩托车', '用料', '研发', '组装']
0.8767674428952017
------------------------------------------------
隆鑫摩托十年内产品质量会有飞跃性的提升吗？
['十年', '隆鑫', '品质', '组装', '培育', '产品质量', '自主', '发展', '提升', '车型', '品牌', '市场', '老王', '隔壁', '慢慢', '摩托', '肯定', '对手', '时间',

In [130]:
vec_store={}
for item1 in item_lst[:20]:
    vec_store.setdefault(item1,{})
    for item2 in list(item_vec_dict.keys()):
        sim=np.dot(item_vec_dict[item1],item_vec_dict[item2])
        vec_store[item1][item2]=sim
    print(item_lst.index(item1))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [34]:
sentences=word2vec.Text8Corpus(r'F:\work\5.18 word2vec训练文章id\data.txt')
model=word2vec.Word2Vec(sentences,size=50,min_count=400,window=5,compute_loss=True,
                  workers=6,sg=0,sample=1e-5,hs=0,negative=40,iter=1,sorted_vocab=1)
essay_list=list(model.wv.vocab.keys())
essay_sim_degree=dict()
for essay in essay_list:
    essay_sim_degree.setdefault(essay,{})
    for x in model.wv.most_similar(essay,topn=50):
        sim=x[0]
        degree=x[1]
        if degree>=0.7:
            essay_sim_degree[essay][sim]=degree
len(essay_sim_degree)

11452

In [37]:
L=[]
for e1 in item_lst[:20]:
    x=essay_sim_degree[str(e1)]
    for e2 in x.keys():
        if e1 in essay_info.keys() and int(e2) in essay_info.keys():
            L.append((e1,essay_info[e1]['title'],essay_info[e1]['content'],essay_info[e1]['keywords'],
                      e2,essay_info[int(e2)]['title'],essay_info[int(e2)]['content'],essay_info[int(e2)]['keywords'],x[e2]))
output = pd.DataFrame(L,columns=['essay_1','essay1_title','essay1_content','essay1_keywords',
                                 'essay_2','essay2_title','essay2_content','essay2_keywords','sim'])
output.sort_values(by=['essay_1','sim'],ascending=[True,False],inplace=True)
output.to_csv(r'F:\work\12.3 文章关键词\1207word2vec.csv',index=False)

In [65]:
L=[]
for e1 in item_lst[:20]:
    x=vec_store[e1]
    x_sorted=sorted(x.items(), key=lambda x:x[1], reverse=True)[:50]
    for k in x_sorted:
        e2=k[0]
        sim=k[1]
        if e1 in essay_info.keys() and e2 in essay_info.keys():
            L.append((e1,essay_info[e1]['title'],essay_info[e1]['content'],essay_info[e1]['keywords'],
                      e2,essay_info[e2]['title'],essay_info[e2]['content'],essay_info[e2]['keywords'],sim))
output = pd.DataFrame(L,columns=['essay_1','essay1_title','essay1_content','essay1_keywords',
                                 'essay_2','essay2_title','essay2_content','essay2_keywords','sim'])
output.sort_values(by=['essay_1','sim'],ascending=[True,False],inplace=True)
output.to_csv(r'F:\work\12.3 文章关键词\1210keywords_sim_tfidf.csv',index=False,encoding='utf_8_sig')